In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

#Importing facts

facts = pd.read_pickle("data/fan-systems-facts.pkl")
facts

,patent_id,sentence_id,head,relation,tail
0,10001020,10001020_0,present invention,generally relates to,fans
1,10001020,10001020_0,present invention,generally relates to,fan assemblies
2,10001020,10001020_0,present invention,generally relates to,fan wheels
3,10001020,10001020_0,present invention,generally relates to,composite fan blades
4,10001020,10001020_0,present invention,generally relates,attachment
...,...,...,...,...,...
2927524,9995315,9995315_37,wire winding space,is increased for,convenient assembling
2927525,9995315,9995315_37,stator,in,motor housing
2927526,9995315,9995315_37,stator,in,cover
2927527,9995315,9995315_37,cover,of,ceiling fan motor


In [3]:
#Specifing the issue
issue = "airflow noise"

#Finding sentences where the issue has been mentioned as entities
sentence_ids = facts.loc[(facts["head"] + " " + facts["tail"]).str.contains(issue)]["sentence_id"].unique().tolist()

#Filtering the facts from sentences thus identified
facts = facts.loc[facts["sentence_id"].isin(sentence_ids)]
facts

,patent_id,sentence_id,head,relation,tail
76938,10100846,10100846_43,unfortunate but unavoidable by product,of,desire
76939,10100846,10100846_43,desire,for,airflow
76940,10100846,10100846_43,“better” fan,demonstrate,higher ratio
76941,10100846,10100846_43,higher ratio,of,airflow
76942,10100846,10100846_43,higher ratio,to,noise production
...,...,...,...,...,...
2854491,9909485,9909485_28,direction,to reduce,airflow loss
2854492,9909485,9909485_28,direction,to reduce,noise
2854493,9909485,9909485_28,recirculating airflow,to reduce,airflow separation
2854494,9909485,9909485_28,recirculating airflow,to reduce,airflow loss


In [4]:
import random

#shuffling the facts
facts["facts"] = facts["head"] + " :: " + facts["relation"] + " :: " + facts["tail"]
fact_list = facts["facts"].tolist()
random.shuffle(fact_list)

#getting the list of facts as text file
with open("data/airflow-facts.txt", "w+", encoding="utf-8") as f:
    f.write("\n".join(fact_list))

In [5]:
#Import the sentences

sentences = pd.read_pickle("data/fan-systems-sentences.pkl")
sentences

,patent_id,sentence_id,sentence
0,10001020,10001020_0,The present invention generally relates to fan...
1,10001020,10001020_1,The improved fan blade is characterized by a f...
2,10001020,10001020_2,The fan blade body includes opposingly paired ...
3,10001020,10001020_3,The fan blade/fan blade body may be a single t...
4,10001020,10001020_4,"As to the latter, it is advantageously contemp..."
...,...,...,...
603179,9995315,9995315_33,The ceiling fan motor housing and cover side f...
603180,9995315,9995315_34,A ceiling fan motor housing and cover side fix...
603181,9995315,9995315_35,The motor housing includes a rotor and plural ...
603182,9995315,9995315_36,The cover includes plural fixing holes formed ...


In [6]:
#Find sentences that include the issue term

sentences = sentences.loc[sentences["sentence"].str.contains(issue)]
sentences

,patent_id,sentence_id,sentence
136651,10718355,10718355_1,"To sum up, in some embodiments of the centrifu..."
136683,10718355,10718355_33,"Therefore, the vortex flow is not synchronized..."
136703,10718355,10718355_53,"By the foregoing structural configuration , th..."
201357,11125238,11125238_68,"With this arrangement, the wake of section 230..."
201371,11125238,11125238_82,"In this way, the trail of section 230 can be i..."
309968,5979595,5979595_13,It is another object of the present invention ...
310090,5979595,5979595_135,"In all cases, the unique radial arrangement of..."
310115,5979595,5979595_160,"In addition, the body member can be equipped w..."
321163,6148954,6148954_13,It is another object of the present invention ...
321284,6148954,6148954_134,"In all cases, the unique radial arrangement of..."


In [7]:
#getting the list of sentences as text file

with open("data/airflow-sentences.txt", "w+", encoding="utf-8") as f:
    f.write("\n\n".join(sentences["sentence"].tolist()))